**Titanic Notebook**

This is a really basic effort. I created this when I was first really getting into machine learning and coding in Python, and I think that this inexperience shows. I read a lot of other notebooks while working this out, and have tried to give credit where it was due. The only really original-ish contribution I have made in this is through some feature engineering...

I assumed that the position of people's cabins on the ship may have influenced their chances of surviving. So I went and looked up some schematics of th Titanic and made a feature based on this info. It wasn't really a game changer in terms of improving the predictive power of my model, but I thought it was an interesting inclusion to the overall body of work on Kaggle for this competition.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
X_test = pd.read_csv("/kaggle/input/titanic/test.csv")
Y_train = train_data['Survived']

In [3]:
import sys
import warnings
warnings.filterwarnings('ignore')

from collections import Counter

from sklearn.model_selection import train_test_split

from sklearn.ensemble import VotingClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier


from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# Removing outliers

Learned from and acknowledgement of: https://www.kaggle.com/ritesh2000/ultimate-titanic-guide-for-beginners#Outlier-Detection

In [4]:
def detect_outliers(df,features):
    outlier_indices = []
    
    for c in features:
        # 1st quartile
        Q1 = np.percentile(df[c],25)
        # 3rd quartile
        Q3 = np.percentile(df[c],75)
        # IQR
        IQR = Q3 - Q1
        # Outlier step
        outlier_step = IQR * 1.5
        # detect outlier and their indeces
        outlier_list_col = df[(df[c] < Q1 - outlier_step) | (df[c] > Q3 + outlier_step)].index
        # store indeces
        outlier_indices.extend(outlier_list_col)
    
    outlier_indices = Counter(outlier_indices)
    multiple_outliers = list(i for i, v in outlier_indices.items() if v > 2)
    
    return multiple_outliers

# X_train.loc[detect_outliers(X_train,["Age","SibSp","Parch","Fare"])]

# drop outliers
train_data = train_data.drop(detect_outliers(train_data,["Age","SibSp","Parch","Fare"]),axis = 0).reset_index(drop = True)
X_test = X_test.drop(detect_outliers(X_test,["Age","SibSp","Parch","Fare"]),axis = 0).reset_index(drop = True)

# Creating 'Title' feature

Learned from and acknowledgment of: https://www.kaggle.com/zohaib123/eda-to-prediction-dietanic

In [5]:
# Creating 'Title'

# https://www.kaggle.com/zohaib123/eda-to-prediction-dietanic

def get_title(in_data): # extract the Titles and cluster them

    in_data['Title']=0
    for i in in_data:
        in_data['Title']=in_data.Name.str.extract('([A-Za-z]+)\.') #lets extract the Salutations

    in_data['Title'].replace(['Mlle','Mme','Ms','Dr','Major','Lady','Countess','Jonkheer','Col'
                             ,'Rev','Capt','Sir','Don', 'Dona']
                                  ,['Miss','Miss','Miss','Mr','Mr','Mrs'
                            ,'Mrs','Other','Other','Other','Mr','Mr','Mr', 'Mrs'],inplace=True)

# call this newly defined function
get_title(train_data)
get_title(X_test)

In [6]:
# Creating 'Shared' to see if people are sharing a cabin

train_data['Shared'] = train_data['Cabin'].dropna().duplicated(False)
train_data['Shared'] = train_data['Shared'].fillna(False)
X_test['Shared'] = X_test['Cabin'].dropna().duplicated(False)
X_test['Shared'] = X_test['Shared'].fillna(False)

In [7]:
#Creating 'Position'

# https://www.encyclopedia-titanica.org/titanic-deckplans/b-deck.html

def find_position(in_data):

    # Creating 'Position'
    in_data.loc[in_data['Cabin'].str.count(' ') > 0, 'Cabin'] = 'X999'
    in_data.loc[in_data['Cabin'].str.len() < 2, 'Cabin'] = 'X999'
    in_data['Cabin'] = in_data['Cabin'].fillna('X999')

    in_data['Cab_No'] = in_data['Cabin'].str[1:].astype(int)
    in_data['Cab_Level'] = in_data['Cabin'].str[0]

    # Level A
    in_data.loc[(in_data['Cab_No'] % 2 == 0) & (in_data['Cab_Level'] == 'A'), 'Position'] = 'Front Left'
    in_data.loc[(in_data['Cab_No'] % 2 != 0) & (in_data['Cab_Level'] == 'A'), 'Position'] = 'Front Right'

    # Level B
    in_data.loc[(in_data['Cab_No'] % 2 == 0) & (in_data['Cab_Level'] == 'B') & (in_data['Cab_No'] < 80), 'Position'] = 'Middle Left'
    in_data.loc[(in_data['Cab_No'] % 2 != 0) & (in_data['Cab_Level'] == 'B') & (in_data['Cab_No'] < 80), 'Position'] = 'Middle Right'
    in_data.loc[(in_data['Cab_No'] % 2 == 0) & (in_data['Cab_Level'] == 'B') & (in_data['Cab_No'] < 50), 'Position'] = 'Front Left'
    in_data.loc[(in_data['Cab_No'] % 2 != 0) & (in_data['Cab_Level'] == 'B') & (in_data['Cab_No'] < 50), 'Position'] = 'Front Right'
    in_data.loc[(in_data['Cab_No'] % 2 == 0) & (in_data['Cab_Level'] == 'B') & (in_data['Cab_No'] > 79), 'Position'] = 'Rear Left'
    in_data.loc[(in_data['Cab_No'] % 2 != 0) & (in_data['Cab_Level'] == 'B') & (in_data['Cab_No'] > 79), 'Position'] = 'Rear Right'

    # Level C
    in_data.loc[(in_data['Cab_No'] % 2 == 0) & (in_data['Cab_Level'] == 'C') & (in_data['Cab_No'] < 91), 'Position'] = 'Middle Left'
    in_data.loc[(in_data['Cab_No'] % 2 != 0) & (in_data['Cab_Level'] == 'C') & (in_data['Cab_No'] < 84), 'Position'] = 'Middle Right'
    in_data.loc[(in_data['Cab_No'] % 2 == 0) & (in_data['Cab_Level'] == 'C') & (in_data['Cab_No'] < 39), 'Position'] = 'Front Left'
    in_data.loc[(in_data['Cab_No'] % 2 != 0) & (in_data['Cab_Level'] == 'C') & (in_data['Cab_No'] < 39), 'Position'] = 'Front Right'
    in_data.loc[(in_data['Cab_No'] % 2 == 0) & (in_data['Cab_Level'] == 'C') & (in_data['Cab_No'] > 90), 'Position'] = 'Rear Left'
    in_data.loc[(in_data['Cab_No'] % 2 != 0) & (in_data['Cab_Level'] == 'C') & (in_data['Cab_No'] > 83), 'Position'] = 'Rear Right'
    in_data.loc[(in_data['Cab_No'] % 2 == 0) & (in_data['Cab_Level'] == 'C') & (in_data['Cab_No'] > 103) & (in_data['Cab_No'] < 141), 'Position'] = 'Middle Left'
    in_data.loc[(in_data['Cab_No'] % 2 != 0) & (in_data['Cab_Level'] == 'C') & (in_data['Cab_No'] > 83) & (in_data['Cab_No'] < 124), 'Position'] = 'Middle Right'

    # Level D
    in_data.loc[(in_data['Cab_No'] % 2 == 0) & (in_data['Cab_Level'] == 'D') & (in_data['Cab_No'] < 51), 'Position'] = 'Front Left'
    in_data.loc[(in_data['Cab_No'] % 2 != 0) & (in_data['Cab_Level'] == 'D') & (in_data['Cab_No'] < 51), 'Position'] = 'Front Right'
    in_data.loc[(in_data['Cab_No'] % 2 == 0) & (in_data['Cab_Level'] == 'D') & (in_data['Cab_No'] > 50), 'Position'] = 'Rear Left'
    in_data.loc[(in_data['Cab_No'] % 2 != 0) & (in_data['Cab_Level'] == 'D') & (in_data['Cab_No'] > 50), 'Position'] = 'Rear Right'

    # Level E
    in_data.loc[(in_data['Cab_Level'] == 'E') & (in_data['Cab_No'] < 70), 'Position'] = 'Middle Right'
    in_data.loc[(in_data['Cab_Level'] == 'E') & (in_data['Cab_No'] < 23), 'Position'] = 'Front Right'
    in_data.loc[(in_data['Cab_Level'] == 'E') & (in_data['Cab_No'] > 69), 'Position'] = 'Rear Right'
    in_data.loc[(in_data['Cab_Level'] == 'E') & (in_data['Cab_No'] > 100), 'Position'] = 'Rear Left'
    in_data.loc[(in_data['Cab_No'] % 2 == 0) & (in_data['Cab_Level'] == 'E') & (in_data['Cab_No'] > 137), 'Position'] = 'Rear Left'
    in_data.loc[(in_data['Cab_No'] % 2 != 0) & (in_data['Cab_Level'] == 'E') & (in_data['Cab_No'] > 138), 'Position'] = 'Rear Right'

    # Level F
    in_data.loc[(in_data['Cab_No'] % 2 == 0) & (in_data['Cab_Level'] == 'F'), 'Position'] = 'Rear Left'
    in_data.loc[(in_data['Cab_No'] % 2 != 0) & (in_data['Cab_Level'] == 'F'), 'Position'] = 'Rear Right'

    # Level G
    in_data.loc[(in_data['Cab_No'] % 2 == 0) & (in_data['Cab_Level'] == 'G'), 'Position'] = 'Rear Left'
    in_data.loc[(in_data['Cab_No'] % 2 != 0) & (in_data['Cab_Level'] == 'G'), 'Position'] = 'Rear Right'

    # Where Cabin was not recorded, or where no Cabin could be determined.
    in_data.loc[in_data['Cab_No'] == 999, 'Position'] = 'Unknown'
    
# call this newly defined function
find_position(train_data)
find_position(X_test)

In [8]:
# Creating 'Family' - a single feature that counts how many family members the person had onboard
# It does this by combining Siblings and Parents.
# What about married couples?

train_data['Family'] = train_data['SibSp'] + train_data['Parch']
X_test['Family'] = X_test['SibSp'] + X_test['Parch']

In [9]:
# Creating 'Group' - which reflects whether the person is travelling alone or not. This is based on Family
# size and the 'Shared' feature.

train_data.loc[((train_data['Shared'] == False) | (train_data['Cabin'] == 'X999')) & (train_data['Family'] <= 1), 'Group'] = 'Alone'
train_data.loc[((train_data['Family'] > 1) & (train_data['Family'] <= 5)), 'Group'] = 'Small'
train_data.loc[((train_data['Shared'] == True) & (train_data['Family'] <= 1)), 'Group'] = 'Small'
train_data.loc[(train_data['Family'] > 5), 'Group'] = 'Large'
X_test.loc[((X_test['Shared'] == False) | (X_test['Cabin'] == 'X999')) & (X_test['Family'] <=1), 'Group'] = 'Alone'
X_test.loc[((X_test['Family'] > 1) & (X_test['Family'] <= 5)), 'Group'] = 'Small'
X_test.loc[((X_test['Shared'] == True) & (X_test['Family'] <= 1)), 'Group'] = 'Small'
X_test.loc[(X_test['Family'] > 5), 'Group'] = 'Large'

In [10]:
# Creating 'Name_Length' - I don't know why this is significant, however multiple analyses have
# shown that it is a relevant differentiator.

train_data.loc[(train_data['Name'].apply(len) <= 30), 'Name_Length'] = 'Short'
train_data.loc[(train_data['Name'].apply(len) > 30), 'Name_Length'] = 'Long'
X_test.loc[(X_test['Name'].apply(len) <= 30), 'Name_Length'] = 'Short'
X_test.loc[(X_test['Name'].apply(len) > 30), 'Name_Length'] = 'Long'

In [11]:
# impute for the column 'Embarked' using the mode, which is 'S'
train_data['Embarked'].fillna('S',inplace=True)

In [12]:
# impute for the column 'Fare' using the mean
X_test["Fare"].fillna(X_test["Fare"].mean(), inplace=True) 

In [13]:
# impute Age based on the median age for that person's Title
train_data['Age'] = train_data.groupby(['Title'])['Age'].apply(lambda x: x.fillna(x.median()))
X_test['Age'] = X_test.groupby(['Title'])['Age'].apply(lambda x: x.fillna(x.median()))

In [14]:
def column_bin_q(in_data, in_col, out_col, div):
    in_data[out_col] = pd.qcut(in_data[in_col], q=div, labels=False)

# call this newly defined function
column_bin_q(train_data, 'Age', 'Age_Bin_Q', 3)
column_bin_q(train_data, 'Fare', 'Fare_Bin_Q', 3)
column_bin_q(X_test, 'Age', 'Age_Bin_Q', 3)
column_bin_q(X_test, 'Fare', 'Fare_Bin_Q', 3)

In [15]:
def column_fact(in_data, in_col, out_col):
    in_data[out_col] = in_data[in_col].factorize()[0]

# call this newly defined function
column_fact(train_data, 'Sex', 'Sex_Factorised')
column_fact(train_data, 'Embarked', 'Embarked_Factorised')
column_fact(train_data, 'Title', 'Title_Factorised')
column_fact(train_data, 'Position', 'Position_Factorised')
column_fact(train_data, 'Cab_Level', 'Cab_Level_Factorised')
column_fact(train_data, 'Group', 'Group_Factorised')
# column_fact(train_data, 'Age_Bin', 'Age_Bin_Factorised')
# column_fact(train_data, 'Fare_Bin', 'Fare_Bin_Factorised')
column_fact(train_data, 'Name_Length', 'Name_Length_Factorised')
column_fact(X_test, 'Sex', 'Sex_Factorised')
column_fact(X_test, 'Embarked', 'Embarked_Factorised')
column_fact(X_test, 'Title', 'Title_Factorised')
column_fact(X_test, 'Position', 'Position_Factorised')
column_fact(X_test, 'Cab_Level', 'Cab_Level_Factorised')
column_fact(X_test, 'Group', 'Group_Factorised')
# column_fact(X_test, 'Age_Bin', 'Age_Bin_Factorised')
# column_fact(X_test, 'Fare_Bin', 'Fare_Bin_Factorised')
column_fact(X_test, 'Name_Length', 'Name_Length_Factorised')

In [16]:
Y_train = train_data['Survived']

train = train_data.drop(['Age', 'Fare', 'Name', 'Shared', 'Sex', 'Ticket', 'Cabin', 'Cab_No', 
                          'Embarked', 'Title', 'Cab_Level', 'Position', 'Group', 
                          'Name_Length', 'Survived',  'SibSp', 'Parch', 'PassengerId'], axis=1)
Xtest = X_test.drop(['Age', 'Fare', 'Name', 'Shared', 'Sex', 'Ticket', 'Cabin', 'Cab_No',
                        'Embarked', 'Title', 'Cab_Level', 'Position', 'Group',
                        'Name_Length', 'PassengerId',  'SibSp', 'Parch'], axis=1)

In [17]:
Train_x, Test_x, Train_y, Test_y = train_test_split(train, Y_train, test_size=0.33, random_state=40)

In [18]:
#I played around with these hyperparameters a fair bit while offline. 
GB = GradientBoostingClassifier(n_estimators= 11, learning_rate=0.5, max_depth=1, random_state=0)

XGB = XGBClassifier(learning_rate =0.1, n_estimators=10, max_depth=2, min_child_weight=5,
                         gamma=0.0025, subsample=1, nthread=12, scale_pos_weight=0.35, seed=0)

LGBM = LGBMClassifier(max_depth = 20, learning_rate = 0.01, num_leaves = 35, n_estimators = 250)

VCLF = VotingClassifier(estimators=[('gb', GB), ('XGB', XGB), ('LGBM', LGBM)], voting='hard')

VCLF = VCLF.fit(Train_x, Train_y)
y_pred = VCLF.predict(Test_x)

matrix = confusion_matrix(Test_y, y_pred)
print(matrix)
print(accuracy_score(Test_y, y_pred))
print(precision_score(Test_y, y_pred))
print(recall_score(Test_y, y_pred))
print(f1_score(Test_y, y_pred))

[23:45:32] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[[167  10]
 [ 36  78]]
0.8419243986254296
0.8863636363636364
0.6842105263157895
0.7722772277227723


In [19]:
Y_pred = VCLF.predict(Xtest)

In [20]:
output = pd.DataFrame({'PassengerId': X_test.PassengerId, 'Survived': Y_pred})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
